# Neuroanatomical validation of whole-brain tractographies

In [ ]:
import shutil
import time
import nrrd
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
import h5py
import csv

from dipy.tracking.local import LocalTracking, ThresholdTissueClassifier
from dipy.tracking.utils import random_seeds_from_mask
from dipy.reconst.dti import TensorModel
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,
                                   auto_response)
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model
from dipy.data import fetch_stanford_hardi, read_stanford_hardi, get_sphere
from dipy.segment.mask import median_otsu
from dipy.viz import actor, window
from dipy.io.image import save_nifti
from dipy.io import read_bvals_bvecs
from dipy.core import gradients
from dipy.tracking.streamline import Streamlines
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, gradient_table_from_bvals_bvecs
from dipy.reconst.dti import fractional_anisotropy
import dipy.reconst.dti as dti
from dipy.tracking import utils

import src.dwi_tools as dwi_tools
import src.nn_helper as nn_helper
import src.tracking as tracking

from dipy.tracking.streamline import values_from_volume
import dipy.align.vector_fields as vfu
from dipy.core.sphere import Sphere
from dipy.core import subdivide_octahedron 
from scipy.spatial import KDTree
from dipy.core import subdivide_octahedron 

from dipy.tracking.local import LocalTracking
from dipy.viz import window, actor
from dipy.viz.colormap import line_colors
from dipy.tracking.streamline import Streamlines, transform_streamlines
from dipy.tracking import metrics
from dipy.tracking.utils import random_seeds_from_mask, seeds_from_mask

import numpy as np
import warnings

import tensorflow as tf
from joblib import Parallel, delayed
import multiprocessing
from keras.models import load_model
from keras.layers import Activation

import importlib
importlib.reload(tracking)
importlib.reload(nn_helper)
import src.tracking as tracking
import tensorflow as tf
from src.nn_helper import swish, squared_cosine_proximity_2
from src.SelectiveDropout import SelectiveDropout
import os 
import warnings
import SimpleITK as sitk
import src.wma_interactor as wma
importlib.reload(wma)

In [ ]:
wmaDir = '/home/nico/Code/wma'
pRoughAff = 'data/ISMRM_CSD_Atlas_Registered_new/LinearTransform_3.h5'
pFineAff = '../wma/registered2019/ismrm_csd_fa015_20mm_noreslicing_mrtrixDenoised_reg/ismrm_csd_fa015_20mm_noreslicing_mrtrixDenoised_reg/output_tractography/itk_txform_ismrm_csd_fa015_20mm_noreslicing_mrtrixDenoised_reg.tfm'

In [ ]:
pStreamlines = 'results_tracking/ISMRM_2015_Tracto_challenge_data///nico/trainingdataNEW/ISMRM_2015_Tracto_challenge_data_denoised_preproc/ismrm_csd_fa015_20mm_noreslicing_mrtrixDenoised_curated.vtk/b1000_raw_sw1.0_3x3x3_ut0_rotateD0_ep0/mlp_single_sqCos2_dx_3_dy_3_dz_3_dd_32_LeakyReLU_feat_512_depth_3_output_3_lr_0.0001_dropout_0_bn_0_unitTangent_0_wz_0--0.979982reslice-False-denoising-False-200st-20mm-fa-0.15.vtk'
#pStreamlines = 'data/ismrm_csd_fa015_20mm_noreslicing_mrtrixDenoised_curated.vtk'
#pStreamlines = 'data/ismrm_csd_fa015_20mm_noreslicing_mrtrixDenoised.vtk'

In [ ]:
pIntermediateResults = os.path.split(pStreamlines)[-1]
pReferenceBundle = 'T_CST.mrml'

load first rough affine transformation matrix ISMRM -> roughATLAS

In [ ]:
affRough = wma.loadTransform(pRoughAff)
affRough[2,3] = -74
affRough

#### load affine roughATLAS -> fineATLAS

In [ ]:
affFine = wma.loadTransform(pFineAff)
affFine

#### project streamlines to atlas coordinate system

In [ ]:
sl = dwi_tools.loadVTKstreamlines(pStreamlines)

In [ ]:
sl1 = transform_streamlines(sl, affRough) # rough transform 
sl2 = transform_streamlines(sl1, affFine) # fine transform

In [ ]:
dwi_tools.saveVTKstreamlines(pStreamlines='%s/%s' % (wmaDir, pIntermediateResults), streamlines=sl2)

## cluster streamlines by wma

In [ ]:
#import subprocess
wmaCmd = "%s/bin/python" % (wmaDir)
wmaParam = "%s/bin/wm_cluster_from_atlas.py %s/%s ORG-800FC-100HCP-minimal-atlas/ %s/clustering_output/" % (wmaDir,wmaDir,pIntermediateResults,wmaDir)
#os.system(wmaCmd + " " + wmaParam)
print(wmaCmd + " " + wmaParam)

copy reference mrml file

In [ ]:
mrmlFrom = "%s/reference_mrml_files/%s" % (wmaDir,pReferenceBundle)
mrmlTo = "%s/clustering_output/%s/%s" % (wmaDir,pIntermediateResults.replace('.vtk',''),pReferenceBundle)

In [ ]:
shutil.copyfile(mrmlFrom, mrmlTo)

## print no. of fibres per CST cluster

In [ ]:
pMRML = wmaDir + "/clustering_output/" + pIntermediateResults.replace('.vtk','') + "/T_CST.mrml"

In [ ]:
print(pIntermediateResults)
res = wma.computeNumberOfStreamlinesForEachCluster(pMRML)

In [ ]:
pResults = 'nav_%s.csv' % (pIntermediateResults.replace('.vtk',''))
with open(pResults, 'a') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow([pReferenceBundle.replace('.mrml','')])
    for k, v in res.items():
        spamwriter.writerow([k, v])